In [4]:
import openai
from dotenv import load_dotenv
import os
import json

In [5]:
load_dotenv(".env")
openai.api_key = os.getenv("OPENAI_API_KEY")

### Extracting email values

In [30]:
messages = [
    {"role": "user", "content": "Send Cobus from humanfirst ai an email asking for the monthly report?"}
  ]

def get_completion(messages):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions= [
    {
      "name": "send_email",
      "description": "Please send an email.",
      "parameters": {
        "type": "object",
        "properties": {
          "to_address": {
            "type": "string",
            "description": "To address for email"
          },          
          "subject": {
            "type": "string",
            "description": "subject of the email"
          },
          "body": {
            "type": "string",
            "description": "Body of the email"
          }
        }
      }
    }
  ],
        function_call="auto",
    )
    arguments = response['choices'][0]['message']['function_call']['arguments']
    arguments_dict = json.loads(arguments)
    return arguments_dict

In [31]:
get_completion(messages)

{'to_address': 'cobus@humanfirst.ai',
 'subject': 'Request for Monthly Report',
 'body': "Hi Cobus,\n\nI hope you're doing well. Could you please send me the monthly report for this month? I need it for the upcoming meeting.\n\nThanks,\n[Your Name]"}

### Classify sentiment (v1)

In [35]:
completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[{"role": "user", "content": "I like it"}],
    functions=[
    {
        "name": "get_sentiment",
        "description": "Categorize the user message sentiment ",
        "parameters": {
            "type": "object",
            "properties": {
                "category": {"type": "string", "enum": ["positive", "negative", "neutral"]},
            },
            "required": ["category"],
        },
    }
],
function_call="auto",
)

In [36]:
reply_content = completion.choices[0]
reply_content

<OpenAIObject at 0x11562f530> JSON: {
  "index": 0,
  "message": {
    "role": "assistant",
    "content": null,
    "function_call": {
      "name": "get_sentiment",
      "arguments": "{\n  \"category\": \"positive\"\n}"
    }
  },
  "finish_reason": "function_call"
}

### Classify sentiment (v2)

In [38]:
messages = [
    {"role": "user", "content": "I like it"}
]

def get_sentiment(messages):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=[
        {
            "name": "get_sentiment",
            "description": "Categorize the user message sentiment ",
            "parameters": {
                "type": "object",
                "properties": {
                    "category": {"type": "string", "enum": ["positive", "negative", "neutral"]},
                },
                "required": ["category"],
            },
        }
    ],
    function_call="auto",
    )
    
    arguments = response['choices'][0]['message']['function_call']['arguments']
    arguments_dict = json.loads(arguments)
    return arguments_dict

get_sentiment(messages)

{'category': 'positive'}


### Return job description plus image url

In [10]:
messages = [
    {"role": "user", "content": "I am a web developer"}
]

def get_job_info(messages):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=[
        {
            "name": "get_job_description",
            "description": "Write a detailed description of the job of the user, assign the job to a category and generate an image file path",
            "parameters": {
                "type": "object",
                "properties": {
                    "job_description": {
                        "type": "string",
                        "description": "Write a description of the Job",
                    },
                    "job_category": {
                        "type": "string", 
                        "enum": ["medizin", "software", "handwerk"],
                        "description": "The job category",
                    },
                    "job_url_path": {
                        "type": "string", 
                        "enum": ["images/medical", "images/software", "images/farming"],
                        "description": "The image path",
                    },
                },
            },
        }],
        function_call={"name": "get_job_description"},
    )
    
    return response

get_job_info(messages)


<OpenAIObject chat.completion id=chatcmpl-7UA6mRybdQls0ZAZHqnpDTVJvO06N at 0x114eb6810> JSON: {
  "id": "chatcmpl-7UA6mRybdQls0ZAZHqnpDTVJvO06N",
  "object": "chat.completion",
  "created": 1687423052,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_job_description",
          "arguments": "{\n  \"job_description\": \"A web developer is responsible for designing, coding, and modifying websites, from layout to function and according to a client's specifications. They strive to create visually appealing sites that feature user-friendly design and clear navigation. Web developers also work closely with clients to understand their needs and create customized solutions that meet their business requirements. Additionally, they use programming languages such as HTML, CSS, JavaScript, and PHP to build and maintain websites. As a web developer, you w

In [45]:
def create_html_image_tag(job_url_path):
    return f'<img src="{job_url_path}" alt="Job image">'

# Usage example
job_url_path = "images/software"  # this value can be extracted from the get_job_info function's response
html_tag = create_html_image_tag(job_url_path)
print(html_tag)


<img src="images/software" alt="Job image">
